In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter


In [3]:
loader = TextLoader("text.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(documents)

In [4]:
docs

[Document(metadata={'source': 'text.txt'}, page_content='Sentence Transformers is a Python framework and model family that uses transformer architectures to create high-quality, fixed-length vector representations (embeddings) of sentences, paragraphs, and even images. These embeddings capture the semantic meaning of the input text, allowing for semantic search, clustering, text similarity, and question-answering by treating similar texts as vectors that are close to each other in a multi-dimensional space. \n\nall-MiniLM-L6-v2\nThis is a sentence-transformers model: It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.\n\nUsage (Sentence-Transformers)\nUsing this model becomes easy when you have sentence-transformers installed:\n\npip install -U sentence-transformers\n\nThen you can use the model like this:\n\nfrom sentence_transformers import SentenceTransformer\nsentences = ["This is an example sentence",

In [5]:
embeddings = OllamaEmbeddings(model="gemma:2b")
db = FAISS.from_documents(docs,embeddings)
db

C:\Users\Bharat\AppData\Local\Temp\ipykernel_31020\3469876571.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


In [16]:
query = "what is sentence transformer"
resp = db.similarity_search(query)
resp[0].page_content

'Intended uses\nOur model is intended to be used as a sentence and short paragraph encoder. Given an input text, it outputs a vector which captures the semantic information. The sentence vector may be used for information retrieval, clustering or sentence similarity tasks.\n\nBy default, input text longer than 256 word pieces is truncated.\n\nTraining procedure\nPre-training\nWe use the pretrained nreimers/MiniLM-L6-H384-uncased model. Please refer to the model card for more detailed information about the pre-training procedure.\n\nFine-tuning\nWe fine-tune the model using a contrastive objective. Formally, we compute the cosine similarity from each possible sentence pairs from the batch. We then apply the cross entropy loss by comparing with true pairs.'

SILILARITY SEARCH WITH SCORE

In [17]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='a23019d1-34a7-43b9-a861-5185dec46f5d', metadata={'source': 'text.txt'}, page_content='Intended uses\nOur model is intended to be used as a sentence and short paragraph encoder. Given an input text, it outputs a vector which captures the semantic information. The sentence vector may be used for information retrieval, clustering or sentence similarity tasks.\n\nBy default, input text longer than 256 word pieces is truncated.\n\nTraining procedure\nPre-training\nWe use the pretrained nreimers/MiniLM-L6-H384-uncased model. Please refer to the model card for more detailed information about the pre-training procedure.\n\nFine-tuning\nWe fine-tune the model using a contrastive objective. Formally, we compute the cosine similarity from each possible sentence pairs from the batch. We then apply the cross entropy loss by comparing with true pairs.'),
  np.float32(4756.0674)),
 (Document(id='cd3f6f73-1b8b-4a8a-ae1e-16283454695a', metadata={'source': 'text.txt'}, page_content='Hyper

In [ ]:
embeddings_vector = embeddings.embed_query(query)
embeddings_vector 

[-0.20736198127269745,
 -0.3027019202709198,
 1.455064058303833,
 2.0424985885620117,
 0.20113542675971985,
 0.8178459405899048,
 -1.1329504251480103,
 -1.6066663265228271,
 0.2144000083208084,
 0.03551708534359932,
 0.06927432864904404,
 0.6502252221107483,
 -0.16748201847076416,
 0.9599049091339111,
 0.29070085287094116,
 0.8698529005050659,
 3.0824508666992188,
 0.9251200556755066,
 0.1335766613483429,
 -0.4765244424343109,
 2.696622133255005,
 -0.8504565954208374,
 -0.0921788364648819,
 -0.33633953332901,
 0.14159145951271057,
 0.26506298780441284,
 -1.9630659818649292,
 0.30688443779945374,
 1.2345951795578003,
 -0.4550548195838928,
 2.7345714569091797,
 1.6164642572402954,
 1.620158314704895,
 0.7625455856323242,
 0.9597045183181763,
 0.4139246642589569,
 -0.694838285446167,
 0.25232577323913574,
 1.450463056564331,
 -0.5555869340896606,
 0.18715646862983704,
 -0.10404540598392487,
 0.822452187538147,
 -0.22259503602981567,
 1.7250196933746338,
 -0.20563623309135437,
 -0.60633325

In [18]:
search_by_vector = db.similarity_search_by_vector(embeddings_vector)
search_by_vector

[Document(id='31f99b1c-437d-49b9-b4c5-1c28df0788fb', metadata={'source': 'text.txt'}, page_content='Background\nThe project aims to train sentence embedding models on very large sentence level datasets using a self-supervised contrastive learning objective. We used the pretrained nreimers/MiniLM-L6-H384-uncased model and fine-tuned in on a 1B sentence pairs dataset. We use a contrastive learning objective: given a sentence from the pair, the model should predict which out of a set of randomly sampled other sentences, was actually paired with it in our dataset.\n\nWe developed this model during the Community week using JAX/Flax for NLP & CV, organized by Hugging Face. We developed this model as part of the project: Train the Best Sentence Embedding Model Ever with 1B Training Pairs. We benefited from efficient hardware infrastructure to run the project: 7 TPUs v3-8, as well as intervention from Googles Flax, JAX, and Cloud team member about efficient deep learning frameworks.'),
 Docume

In [19]:
### Save and load
db.save_local("faiss.index")

In [24]:
new_db = FAISS.load_local("faiss.index", embeddings, allow_dangerous_deserialization=True)

docs = new_db.similarity_search(query)

In [21]:
docs

[Document(id='a23019d1-34a7-43b9-a861-5185dec46f5d', metadata={'source': 'text.txt'}, page_content='Intended uses\nOur model is intended to be used as a sentence and short paragraph encoder. Given an input text, it outputs a vector which captures the semantic information. The sentence vector may be used for information retrieval, clustering or sentence similarity tasks.\n\nBy default, input text longer than 256 word pieces is truncated.\n\nTraining procedure\nPre-training\nWe use the pretrained nreimers/MiniLM-L6-H384-uncased model. Please refer to the model card for more detailed information about the pre-training procedure.\n\nFine-tuning\nWe fine-tune the model using a contrastive objective. Formally, we compute the cosine similarity from each possible sentence pairs from the batch. We then apply the cross entropy loss by comparing with true pairs.'),
 Document(id='cd3f6f73-1b8b-4a8a-ae1e-16283454695a', metadata={'source': 'text.txt'}, page_content='Hyper parameters\nWe trained our 

In [23]:
docs[0].page_content

'Intended uses\nOur model is intended to be used as a sentence and short paragraph encoder. Given an input text, it outputs a vector which captures the semantic information. The sentence vector may be used for information retrieval, clustering or sentence similarity tasks.\n\nBy default, input text longer than 256 word pieces is truncated.\n\nTraining procedure\nPre-training\nWe use the pretrained nreimers/MiniLM-L6-H384-uncased model. Please refer to the model card for more detailed information about the pre-training procedure.\n\nFine-tuning\nWe fine-tune the model using a contrastive objective. Formally, we compute the cosine similarity from each possible sentence pairs from the batch. We then apply the cross entropy loss by comparing with true pairs.'